In [3]:
# data analysis and wrangling
import pandas as pd
import numpy as np
import random as rnd


In [41]:
missing_values = ["n/a", "na", "--",".","NaN"]
df_t_list = []
df_p_list = []
for index in range (3,7):
    df_in = pd.read_csv("Data/training201{}.csv".format(index), na_values = missing_values)  
    df_in2 = pd.read_csv("Data/prediction201{}.csv".format(index), na_values = missing_values)
    
    df_in2 = df_in2.drop(["CHGENROL","FRACDEGRE","RETPCNT"],axis = 1)
    ##drop prediction rows from predictions
    df_t_list.append(df_in)
    df_p_list.append(df_in2)

#read the example submission in
example_submission = pd.read_csv("Data/ex_submit.csv")
submission_col = example_submission.columns 
submission_shape = example_submission.shape

# inpute missing values

In [42]:
for index, df in enumerate(df_t_list):
    df_t_list[index] = df.apply(lambda x: x.fillna(x.mean()),axis=0)
for index, df in enumerate(df_p_list):
    df_p_list[index] = df.apply(lambda x: x.fillna(x.mean()),axis=0)
    

# Seperate the data into X and y and X_p

In [63]:
target_cols = ["CHGENROL","FRACDEGRE","RETPCNT"]
X_list = []
X_p_list = []
y_list = []
#drop the seperate the data into targets and data


for index, (df_t, df_p) in enumerate(zip(df_t_list,df_p_list)):
    cols = [col for col in df_t_list[index].columns if col not in target_cols]
    X_list.append(df_t[cols])
    y_list.append(df_t[target_cols])
    X_p_list.append(df_p)


In [65]:
from sklearn.preprocessing import MinMaxScaler

for index, X in enumerate(X_list):
    minmax = MinMaxScaler()
    X_list[index] = minmax.fit_transform(X)
    
for index, X in enumerate(X_p_list):
    minmax = MinMaxScaler()
    X_p_list[index] = minmax.fit_transform(X)

"""
robust_scaler = preprocessing.RobustScaler()
np_scaled = robust_scaler.fit_transform(dataset[collist[1:]])

np_scaled = dataset[collist[1:]].values
fill_Nan = SimpleImputer(missing_values = np.nan, strategy ="constant", fill_value = -50.0)
np_imp = fill_Nan.fit_transform(np_scaled)
dataset_scaled = pd.DataFrame(np_imp)

# create the Labelencoder object
le = preprocessing.LabelEncoder()
dataset["File"] = le.fit_transform(dataset["File"])
dataset["File"].unique()

#merge the two dataframes
dataset_scaled["33"] = dataset["File"]

##Handle the prediction data

##robust_scaler2 = preprocessing.RobustScaler()
##np_scaled2 = robust_scaler.fit_transform(dataset2[collist[1:]])
np_scaled2 = dataset2[collist[1:]].values

fill_Nan2 = SimpleImputer(missing_values = np.nan, strategy ="constant", fill_value = -50.0)
np_imp2 = fill_Nan2.fit_transform(np_scaled2)
dataset_scaled2 = pd.DataFrame(np_imp2)

# create the Labelencoder object
le2 = preprocessing.LabelEncoder()
dataset2["File"] = le2.fit_transform(dataset2["File"])
dataset2["File"].unique()

#merge the two dataframes
dataset_scaled2["33"] = dataset2["File"]
dataset_scaled2.head(4)

"""

C:\Users\Robert\.conda\envs\sklearn\lib\site-packages\sklearn\preprocessing\data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
C:\Users\Robert\.conda\envs\sklearn\lib\site-packages\sklearn\preprocessing\data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
C:\Users\Robert\.conda\envs\sklearn\lib\site-packages\sklearn\preprocessing\data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
C:\Users\Robert\.conda\envs\sklearn\lib\site-packages\sklearn\preprocessing\data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


'\nrobust_scaler = preprocessing.RobustScaler()\nnp_scaled = robust_scaler.fit_transform(dataset[collist[1:]])\n\nnp_scaled = dataset[collist[1:]].values\nfill_Nan = SimpleImputer(missing_values = np.nan, strategy ="constant", fill_value = -50.0)\nnp_imp = fill_Nan.fit_transform(np_scaled)\ndataset_scaled = pd.DataFrame(np_imp)\n\n# create the Labelencoder object\nle = preprocessing.LabelEncoder()\ndataset["File"] = le.fit_transform(dataset["File"])\ndataset["File"].unique()\n\n#merge the two dataframes\ndataset_scaled["33"] = dataset["File"]\n\n##Handle the prediction data\n\n##robust_scaler2 = preprocessing.RobustScaler()\n##np_scaled2 = robust_scaler.fit_transform(dataset2[collist[1:]])\nnp_scaled2 = dataset2[collist[1:]].values\n\nfill_Nan2 = SimpleImputer(missing_values = np.nan, strategy ="constant", fill_value = -50.0)\nnp_imp2 = fill_Nan2.fit_transform(np_scaled2)\ndataset_scaled2 = pd.DataFrame(np_imp2)\n\n# create the Labelencoder object\nle2 = preprocessing.LabelEncoder()\nd

In [73]:
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV

In [82]:
def MLP_model(X, y):
    MLP = MLPRegressor(learning_rate='adaptive', max_iter=1000, random_state=4, warm_start = True)
    params = {
    "solver":["lbfgs","adam"],
    'activation': ['logistic', 'relu', 'tanh'],  # <-- added 'tanh' as third non-linear activation function
    'alpha': np.logspace(0.0001, 100, 10),
    'hidden_layer_sizes': [
        (10, 10), (20, 10), (30, 10),
        (40, 10), (90, 10), (90, 30, 10),(100,10,10,10),(30,20,10),(60,30,30,30,3)  # <-- added more neurons or layers
    ]
    }
    clf = GridSearchCV(estimator=MLP, param_grid=params,
                   n_jobs=-1, cv=3, refit=True, verbose=5) 
    
    clf.fit(X, y)
   
    return clf, clf.best_params_

In [ ]:
import joblib
clf_list = []
for X , y in zip(X_list,y_list):
    clf, best_params = MLP_model(X,y)
    clf_list.append((clf,best_params))


Fitting 3 folds for each of 540 candidates, totalling 1620 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 36 concurrent workers.
exception calling callback for <Future at 0x211926896d8 state=finished raised TerminatedWorkerError>
Traceback (most recent call last):
  File "C:\Users\Robert\.conda\envs\sklearn\lib\site-packages\sklearn\externals\joblib\externals\loky\_base.py", line 625, in _invoke_callbacks
    callback(self)
  File "C:\Users\Robert\.conda\envs\sklearn\lib\site-packages\sklearn\externals\joblib\parallel.py", line 309, in __call__
    self.parallel.dispatch_next()
  File "C:\Users\Robert\.conda\envs\sklearn\lib\site-packages\sklearn\externals\joblib\parallel.py", line 731, in dispatch_next
    if not self.dispatch_one_batch(self._original_iterator):
  File "C:\Users\Robert\.conda\envs\sklearn\lib\site-packages\sklearn\externals\joblib\parallel.py", line 759, in dispatch_one_batch
    self._dispatch(tasks)
  File "C:\Users\Robert\.conda\envs\sklearn\lib\site-packages\sklearn\externals\joblib\parallel.py", line 716, in _disp

In [81]:
clf_list

[]

C:\Users\Robert\.conda\envs\sklearn\lib\site-packages\sklearn\externals\joblib\externals\loky\backend\utils.py:55: UserWarning: Failed to kill subprocesses on this platform. Pleaseinstall psutil: https://github.com/giampaolo/psutil
  warnings.warn("Failed to kill subprocesses on this platform. Please"
Exception in thread QueueManagerThread:
Traceback (most recent call last):
  File "C:\Users\Robert\.conda\envs\sklearn\lib\threading.py", line 917, in _bootstrap_inner
    self.run()
  File "C:\Users\Robert\.conda\envs\sklearn\lib\threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Robert\.conda\envs\sklearn\lib\site-packages\sklearn\externals\joblib\externals\loky\process_executor.py", line 674, in _queue_management_worker
    recursive_terminate(p)
  File "C:\Users\Robert\.conda\envs\sklearn\lib\site-packages\sklearn\externals\joblib\externals\loky\backend\utils.py", line 28, in recursive_terminate
    _recursive_terminate_without_psutil(proces

In [79]:
predict_raw2 = rfr2.predict(X_p)

[Parallel(n_jobs=36)]: Using backend ThreadingBackend with 36 concurrent workers.
[Parallel(n_jobs=36)]: Done  90 tasks      | elapsed:    0.0s
[Parallel(n_jobs=36)]: Done 293 tasks      | elapsed:    0.0s
[Parallel(n_jobs=36)]: Done 576 tasks      | elapsed:    0.0s
[Parallel(n_jobs=36)]: Done 941 tasks      | elapsed:    0.1s
[Parallel(n_jobs=36)]: Done 1386 tasks      | elapsed:    0.2s
[Parallel(n_jobs=36)]: Done 1913 tasks      | elapsed:    0.3s
[Parallel(n_jobs=36)]: Done 2000 out of 2000 | elapsed:    0.3s finished


In [89]:

y_p1 = pd.DataFrame(predict_raw)
y_p2 = pd.DataFrame(predict_raw2)

y_p2.describe()

,0,1,2
count,1188.000000,1188.000000,1188.000000
mean,1.001158,0.203068,74.493330
std,0.016778,0.031128,9.463537
min,0.914500,0.091587,43.991000
25%,0.994055,0.185336,67.193375
50%,1.000659,0.199732,74.045750
75%,1.007852,0.218781,80.908500
max,1.166484,0.346628,97.386500


In [101]:
file_labes = dataset["File"]

In [102]:
p_2013_np = predict_raw2[file_labels[file_labels == 0]]
p_2014_np = predict_raw2[file_labels[file_labels == 1]]
p_2015_np = predict_raw2[file_labels[file_labels == 2]]
p_2016_np = predict_raw2[file_labels[file_labels == 2]]

In [123]:
nuid = dataset2["NUID"].values
nuid = nuid[:297]
nuid= nuid.reshape(297,1)

In [159]:
k = 600

In [160]:
p_all = np.hstack((nuid,p_2013_np, p_2014_np, p_2015_np, p_2016_np))
p_df = pd.DataFrame(p_all)
#test if same nuid
#p_df[0][k%297]

9577.0

In [163]:
example_sub = pd.read_csv("Data/ex_submit.csv")
p_df.columns = example_sub.columns 
#test of same nuid
#dataset2["NUID"][k]



In [166]:
export_csv = p_df.to_csv (r'predictions1.csv', index = None, header=True)

In [167]:
export_csv

np_scaled2[:,26:29] = predict_raw
np_unscaled2 = robust_scaler.inverse_transform(np_scaled2)

y_p_unsc = np_unscaled2[:,26:29]

y_p_unsc

In [12]:
#y_p = predictions

In [13]:
#y_t = y.values
#y_t

In [14]:
#y_t.shape

In [15]:
#y_p.shape

In [16]:
#def RMSE(targets, predictions):
 #   return np.sqrt(np.mean((predictions-targets)**2))

RMSE(y_t,y_p)

""""""np_scaled_2 = np_scaled.copy()
np_unscaled = robust_scaler.inverse_transform(np_scaled)

np_scaled_2[:,26:29] = y_p
np_unscaled_p = robust_scaler.inverse_transform(np_scaled_2)

y_t_unsc = np_unscaled[:,26:29]
#y_p_unsc = np_unscaled_p[:,26:29]
#y_p_unsc[0:10,:]

y_t_unsc

y_p_unsc

RMSE(y_t_unsc, y_p_unsc)